In [ ]:
inputPath = "/user/s292129/data/exam_ex2_data/Stocks_Prices.txt"
outputPath1 = "res_out_ExExemple4_1/"
outputPath2 = "res_out_ExExemple4_2/"

In [ ]:
inputRDD = sc.textFile(inputPath)

# Task A

In [ ]:
inputRDD = inputRDD.filter(lambda line: line.split(",")[1].startswith("2017"))

In [ ]:
stockIDDate_pricesRDD = inputRDD.map(lambda line: ((line.split(',')[0],\
                                                    line.split(',')[1]),\
                                                    line.split(',')[3]))

In [ ]:
key_maxminRDD = stockIDDate_pricesRDD.combineByKey(lambda e: (e, e),\
            lambda c, e: (max(c[0],e), min(c[1],e)),\
            lambda c1, c2: (max(c1[0], c2[0]), min(c1[1],c2[1])) )

In [ ]:
key_maxminRDD.collect()

In [ ]:
key_dailyVariationRDD = key_maxminRDD.mapValues(lambda v: float(v[0]) - float(v[1])).cache()

In [ ]:
key_dailyVariationRDD.collect()

In [ ]:
filteredRDD = key_dailyVariationRDD.filter(lambda pair: pair[1] > 10)

In [ ]:
stockID_dateRDD = filteredRDD.map(lambda pair: (pair[0][0], 1))

In [ ]:
stockID_dateRDD.collect()

In [ ]:
output1RDD = stockID_dateRDD.reduceByKey(lambda v1, v2: v1 + v2)

In [ ]:
output1RDD.saveAsTextFile(outputPath1)

# Task B - ERRATO

In [ ]:
# This funciton is already provided
from datetime import datetime, timedelta

def previousDate(mydate):
    currentDate = datetime.strptime(mydate, "%Y/%m/%d")   
    prevDate=currentDate - timedelta(days=1)
    
    return prevDate.strftime("%Y/%m/%d")  

In [ ]:
key_dailyVariationRDD = key_dailyVariationRDD.map(lambda line: [line.split(",")[0], line.split(",")[1], float(line.split(",")[2])])

In [ ]:
key_dailyVarDF = spark.createDataFrame(key_dailyVariationRDD, ["stockID","date","dailyVar"])

In [ ]:
key_dailyVarDF.createOrReplaceTempView("variations")

In [ ]:
copiedDF = key_dailyVarDF

In [ ]:
copiedDF.withColumnRenamed("date","date2").withColumnRenamed("stockID","stockID2").createOrReplaceTempView("variations_copy")

In [ ]:
returnedDF = spark.sql("SELECT * \
                       FROM variations JOIN variations_copy ON stockID=stockID2 AND previousDate(date)=date2")

In [ ]:
returnedDF.show()

# Task B - ALTERNATIVE WITH RDD - ERRATO

In [ ]:
key_dailyVariationRDD_copy = key_dailyVariationRDD
jointRDD = key_dailyVariationRDD.join(key_dailyVariationRDD_copy)

def mapFunct(pair):
    stockID = pair[0]
    date1 = pair[1][0].split(",")[0]
    dailyVar1 = pair[1][0].split(",")[1]
    date2 = pair[1][1].split(",")[0]
    dailyVar2 = pair[1][1].split(",")[1]
    if (previousDate(date1) == date2 and abs(dailyVar1 - dailyVar2) <= 0.1):
        return (stockID, date2)
        
    
outputRDD = jointRDD.map(mapFunct)


In [ ]:
outputRDD.collect()